In [ ]:
import numpy as np
import sys
from functools import partial
import matplotlib.pyplot as plt
from scipy.stats import special_ortho_group as sp
plt.rcParams["figure.figsize"] = (10,10)
from matplotlib import ticker, cm
import matplotlib.colors as mcolor
from matplotlib.ticker import LogFormatter, LogLocator

In [ ]:
def gradient_descent(gradient, start, learn_rate, iterations):
    vec = start
    for _ in range(iterations):
        vec += -learn_rate * gradient(vec)
    return vec

In [ ]:
gradient_descent(gradient=lambda v: np.array([2 * v[0], 4 * v[1]**3]),
                       start=np.array([1.0, 1.0]),
                       learn_rate=0.05,
                       iterations=10)

In [ ]:
def dichotomy(fn, a, b, err):

    while True:
      if b - a < err:
        break
      delta = (b - a) / 6
      x1 = (a + b) / 2 - delta
      x2 = (a + b) / 2 + delta
      fx1, fx2 = fn(x1), fn(x2)
      if fx1 > fx2:
        a, b = x1, b
      elif fx1 < fx2:
        a, b = a, x2
      else:
        break
    return b

def gradient_descent_by_error(function, start, learn_rate, error, points):
    def gradient(vector):
        calculated_gradient = np.zeros(vector.size)
        for coord in range(0, error.size):
            coord_offset = np.zeros(vector.size)
            coord_offset[coord] = error[coord]
            calculated_gradient[coord] = (function(vector + coord_offset) - function(vector)) / error[coord]
        return calculated_gradient
    vec = start
    oldvec = []
    while True:
        oldvec = vec.copy()
        vec += -learn_rate * gradient(vec)
        if (np.absolute(vec - oldvec) < error).all():
            break
        if(len(points) == 4000):
            break
        points.append(vec.copy())

    return vec
def gradient_descent_with_dichotomy_by_error(function, start, error, points):
    def gradient(vector):
        calculated_gradient = np.zeros(vector.size)
        for coord in range(0, error.size):
            coord_offset = np.zeros(vector.size)
            coord_offset[coord] = error[coord]
            calculated_gradient[coord] = (function(vector + coord_offset) - function(vector)) / error[coord]
        return calculated_gradient
    vec = start
    while True :
        learn_rate_func = lambda lmbd: function(vec - lmbd * gradient(vec))
        learn_rate = dichotomy(learn_rate_func, 0, 0.5, 10**-5)
        oldvec = vec.copy();
        vec += -learn_rate * gradient(vec)
        if (np.absolute(vec - oldvec) < error).all():
            break
        points.append(vec.copy())
        if(len(points)== 4000):
            break
    return vec

In [ ]:
def gradient_descent_wolfe(function, start, c1, c2, error, points):
    def gradient(vector):
        calculated_gradient = np.zeros(vector.size)
        for coord in range(0, error.size):
            coord_offset = np.zeros(vector.size)
            coord_offset[coord] = error[coord]
            calculated_gradient[coord] = (function(vector + coord_offset) - function(vector)) / error[coord]
        return calculated_gradient
    def calc_rate(vector):
        def phi(a):
            return function(vector - a * gradient(vector))                   
        def derivative_phi(point):
            return (phi(point + 10**-4) - phi(point)) / 10**-4
        def zoom(lo, hi):
            a = hi
            alast_ = 10
            while True:
                alast_ = a
                a = abs(hi - lo) / 2
                if abs(a - alast_) <= 10**-4:
                    break
                ret = a;
                if phi(a) > phi(0) + c1 * a * derivative_phi(0) or phi(a) >= phi(lo):
                    hi = a
                else:
                    if abs(derivative_phi(a)) <= -c2 * derivative_phi(0):
                        ret = a
                        break
                    if derivative_phi(a)*(hi - lo) >= 0:
                        hi = lo
                    lo = a
            return ret
                    
        a0 = 0
        amax = 2
        alast = a0
        a = (a0 + amax) / 2
        res = a
        i = 1
        while True:
            if (abs(a - alast) <= 10**-4):
                break
            if phi(a) > phi(0) + c1 * a * derivative_phi(0) or (phi(a) >= phi(alast) and i > 1):
                res = zoom(alast, a)
                break
            i += 1;

            if abs(derivative_phi(a)) <= -c2 * derivative_phi(0):
                res = a
                break
            if derivative_phi(a) >= 0:
                res = zoom(a, alast)
                break
            alast = a
            a = (a + amax)/2
        return res
            
    vec = start
    while True:
        learn_rate = calc_rate(vec.copy())
        oldvec = vec.copy()
        print(learn_rate)
        vec += -learn_rate * gradient(vec)
        if (np.absolute(vec - oldvec) < error).all():
          break
        points.append(vec.copy())

In [ ]:
plt.rcParams["figure.figsize"] = (12,10)

def draw(type,function,d,points,xlim,ylim,label):
  t = np.linspace(-d,d, 5000)
  X, Y = np.meshgrid(t, t)
  if(type ==0):
      print("For gradient with constant  \n")
  elif(type==1):
      print("For gradient with dihotomy  \n")
  elif(type==1):
      print("For gradient with wolf condition  \n")
  print(len(points))
  fig, ax = plt.subplots()
  plt.xlim(xlim)
  plt.ylim(ylim)
  grad=ax.plot([i[0] for i in points], [i[1] for i in points], 'o-', color="orange",label=label)
  plt.legend(loc="upper right")
  sor= sorted([function([p[0], p[1]]) for p in points])
  cs = ax.contourf(X, Y, function([X, Y]), levels=sor,   norm=mcolor.LogNorm(),cmap='RdBu')
  cbar = fig.colorbar(cs,ticks = LogLocator(subs=range(5)))
  plt.ylabel("y")
  plt.xlabel("x")
  plt.show()
def draw_by_single(function,d, startpoint, xlim, ylim,
                   include_wolf):
  learn_rate=0.05
  points = [startpoint]
  err=np.array([10**-4, 10**-4])
  gradient_descent_with_dichotomy_by_error(function=function,
                                start=startpoint.copy(),
                                error = err,
                                points=points)
  draw(1,function,d,points,xlim,ylim,"Градиентный спуск с дихотомией")
  points_c = [startpoint]
  gradient_descent_by_error(function=function,start=startpoint.copy(),
                            learn_rate=learn_rate,error=err,points=points_c)
  draw(0,function,d,points_c,xlim,ylim,f"Градиентный спуск с постоянным шагом: {learn_rate}")
  if(include_wolf):
      a1=10**-4
      a2= 0.1 
      points_d =[]
      gradient_descent_wolfe(function, startpoint, a1, a2, err, points_d)
      draw(0,function,d,points_d,xlim,ylim,f"Градиентный спуск с условием Вольфа: с1 ={a1}, c2 = {a2}")
def draw_both(function,d, startpoint, xlim, ylim):
  learn_rate=0.075
  err=np.array([10**-4, 10**-4])
  points = [startpoint]
  points_c = [startpoint]
  gradient_descent_with_dichotomy_by_error(function=function,
                                start=startpoint.copy(),
                                error = err,
                                points=points)
  gradient_descent_by_error(function=function,start=startpoint.copy(),
                            learn_rate=learn_rate,error=err,points=points_c)
  t = np.linspace(-d,d, 1000)
  X, Y = np.meshgrid(t, t)
  print("For gradient with dihotomy  \n")
  print(len(points))
  print("For gradient with constant \n")
  print(len(points_c))
  fig, ax = plt.subplots()
  plt.xlim(xlim)
  plt.ylim(ylim)
  ax.plot([i[0] for i in points_c], [i[1] for i in points_c], 'o-',color="green",label =f"Градиентный спуск с постоянным шагом: {learn_rate}")
  ax.plot([i[0] for i in points], [i[1] for i in points], 'o-',color="orange",label = f"Градиентный спуск с дихотомией")
  ax.legend(loc="upper right")
  sor= sorted(list(set([function([p[0], p[1]]) for p in points_c] +[function([p[0], p[1]]) for p in points] )))
  cs = ax.contourf(X, Y, function([X, Y]), levels=sor,   norm=mcolor.LogNorm(),cmap='RdBu')
  cbar = fig.colorbar(cs,ticks =LogLocator(subs=range(8)))
  plt.ylabel("y")
  plt.xlabel("x")
  plt.show()

In [ ]:
def g (vector):
    x = vector[0] 
    y = vector[1]
    return x**2 + y**2 
draw_by_single(g,15, np.array([-2.5, 2.2]), [-4., 4.], [-4.,4.],False)

In [ ]:
def f (vector):
    x = vector[0] 
    y = vector[1]
    return (x+2*y -7)**2 +(2*x+y-5)**2
draw_both(f, 8, np.array([3., -2.5]),[-5.0, 5.0], [-5.0, 5.0])

In [ ]:
def q (vector):
    # return -vector[0]**2 + vector[1]**2  + vector[1]
    x = vector[0] 
    y = vector[1]
    return 0.26*(x**2 + y**2) +0.48*x*y
    # return x*y
draw_both(q,2, np.array([-1., -0.5]), [-1.5, 2.], [-2.,2.])

In [ ]:
def generate_quadratic_form(n,k):
  coef = np.random.randint(2,10)
  diagonal = np.diag(np.random.randint(k,k*coef,n))
  diagonal[0][0] = coef
  diagonal[1][1] = k*coef
  R = sp.rvs(n) 
  return R @ diagonal @ np.linalg.inv(R) ;


In [ ]:
data = []
print("test")
indexes = [2] +[x for x in range(5,52,5)]
def test_nk(n,k):
    a=0
    for t in range(100):
        matrix = generate_quadratic_form(n,k);
        def fun(v):
            ans=0.0
            for i in range(n):
                for j in range(n):
                    ans+=(matrix[i][j]* v[i]*v[j])
            return ans 
        points = []
        gradient_descent_with_dichotomy_by_error(function=fun,
                                                    start=np.array([0.1 for _ in range(n)]),
                                                    error = np.array([10**-6 for i in range(n)]),
                                                     points=points)
        a+=len(points)
    a = a//100
    print(a)
    data.append(a)
    a=0  



In [ ]:
for k in indexes:
    test_nk(2,k)
plt.plot(indexes,data)
plt.ylabel("Количество итераций")
plt.xlabel("Число обусловенности")
plt.show()

In [ ]:
dimensions = [2,5,10,50,100,200]
data=[]
for n in dimensions:
    test_nk(n,5)
plt.plot(dimensions,data)
plt.ylabel("Количество итераций")
plt.xlabel("Размерность пространства")
plt.show()

In [ ]:
def r(vector):
    x = vector[0] 
    y = vector[1]
    return - x*y+ x**(2)+16 *y**(2)
    # return x*y
draw_by_single(r,15, np.array([-0.5, 1.0]), [-0.5, 0.6], [-0.5,0.6],True)

In [ ]:
def r_scaled(vector):
    x = vector[0] 
    y = vector[1]
    return - x*(y/4)+ x**(2)+16 *(y/4)**(2)
    # return x*y
draw_by_single(r_scaled,15, np.array([-0.5, 1.0]), [-0.5, 0.6], [-0.5,0.6],True)

In [ ]:
x_st=[-2.0,-1.5,-1.0,-0.5, 0. ,0.5,1.0,1.5,2.0]
y_st=[-2.0,-1.5,-1.0,-0.5, 0. ,0.5,1.0,1.5,2.0]
def calc_iterations(function):
  for i in x_st:
        for j in y_st:
            if i == 0. and j == 0.:
              continue
            points_d = []
            gradient_descent_by_error(function=function,
                                start=np.array([i, j]),
                                learn_rate=0.05,
                                error = np.array([10**-4, 10**-4]),
                                points=points_d)
            print("x: "+ str(i) +" y: " + str(j) + " iters: " + str(len(points_d)))
            # draw_by_single(func,15, np.array([i, j]), [-2, 2], [-2,2])
  

In [ ]:
calc_iterations(r)

In [ ]:
func = [f,g,r]
x_st=[-2.0,-1.5,-1.0,-0.5, 0. ,0.5,1,1.5,2.0]
y_st=[-2.0,-1.5,-1.0,-0.5, 0. ,0.5,1,1.5,2.0]
def gen_start_point(func):
    for i in x_st:
        for j in y_st:
            if i == 0. and j == 0.:
              continue
            print("x: "+ str(i) +" y: " + str(j))
            draw_by_single(func,15, np.array([i, j]), [-2, 2], [-2,2],False)

In [ ]:
gen_start_point(r)

In [ ]:
gen_start_point(g)